In [2]:
from gensim.models import KeyedVectors

# Load pre-trained vectors directly from the file
model = KeyedVectors.load_word2vec_format('Datasets/en_wiki_word2vec_300.txt', binary=False)

In [3]:
len(nlp.vocab)

1340241

In [4]:
model.similarity('simple', 'impossible')

0.34854212

In [5]:
!pip install tensorflow
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [6]:
df = pd.read_csv('Datasets/IMDB.csv')
df.replace(['positive', 'negative'], [1, 0], inplace=True)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
def preprocess(s):
    '''Function for data pre-processing'''
    # Removing html tags
    TAG_RE = re.compile(r'<[^>]+>')
    sentence = TAG_RE.sub('', s)
    
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [8]:
X = []
sentences = list(df['review'])
for sentence in sentences:
    X.append(preprocess(sentence))

y = df['sentiment']

In [9]:
# Create train-test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(len(X_train), len(X_test), len(y_train), len(y_test))

40000 10000 40000 10000


In [10]:
# Convert into numpy arrays for processing with tensorflo
y_train = np.array(y_train)
y_test = np.array(y_test)
print(type(X_train), type(X_test), type(y_train), type(y_test))

<class 'list'> <class 'list'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [11]:
# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [12]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 300

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [13]:
embedding_matrix = np.zeros((len(model.wv.vocab), 300)) # vector_dim =100
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

c:\users\rojin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
c:\users\rojin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
c:\users\rojin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

# Text Classification with a Simple Neural Network
vocab_size = len(model.wv.vocab)
model = Sequential()
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

c:\users\rojin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()


In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 300)          448487100 
_________________________________________________________________
flatten_1 (Flatten)          (None, 90000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 90001     
Total params: 448,577,101
Trainable params: 90,001
Non-trainable params: 448,487,100
_________________________________________________________________
None


In [17]:
# Train the model
history = model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

Train on 32000 samples, validate on 8000 samples
Epoch 1/10
32000/32000 [==============================] - 6s 178us/step - loss: 0.7515 - acc: 0.5511 - val_loss: 0.6749 - val_acc: 0.6046
Epoch 2/10
32000/32000 [==============================] - 6s 174us/step - loss: 0.5667 - acc: 0.7049 - val_loss: 0.6675 - val_acc: 0.6175
Epoch 3/10
32000/32000 [==============================] - 6s 202us/step - loss: 0.4912 - acc: 0.7645 - val_loss: 0.6423 - val_acc: 0.6364
Epoch 4/10
32000/32000 [==============================] - 6s 202us/step - loss: 0.4265 - acc: 0.8126 - val_loss: 0.7880 - val_acc: 0.5918
Epoch 5/10
32000/32000 [==============================] - 6s 199us/step - loss: 0.3910 - acc: 0.8324 - val_loss: 0.6729 - val_acc: 0.6338
Epoch 6/10
32000/32000 [==============================] - 6s 198us/step - loss: 0.3414 - acc: 0.8693 - val_loss: 0.6721 - val_acc: 0.6461
Epoch 7/10
32000/32000 [==============================] - 6s 195us/step - loss: 0.3301 - acc: 0.8678 - val_loss: 0.6908 - v

In [18]:

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=1)
print("\nTest Accuracy:", score[1])

10000/10000 [==============================] - 2s 157us/step

Test Accuracy: 0.6444000005722046
